https://github.com/neowizard2018/neowizard/tree/master/DeepLearningProject

In [ ]:
# 구글 colab와 구글 drive를 연동(마운트)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil  # 파일과 폴더를 이동하거나 복사하고 싶을 때 사용

shutil.copy("/content/drive/MyDrive/DataSet/Pill/pill_data_croped.Zip", "/content/")  # os.path.join : 경로(패스)명 조작에 관한 처리를 모아둔 모듈

In [ ]:
ROOT_DIR = '/content'
ORG_DATA_DIR = '/content/org_data'
PILL_DATA_DIR = '/content/PILL_data'

In [ ]:
# 압축 풀기
import os
import zipfile

target_file = zipfile.ZipFile("/content/pill_data_croped.Zip", 'r')

target_file.extractall(ORG_DATA_DIR)

In [ ]:
from glob import glob
``
DIR_LIST = glob(ORG_DATA_DIR+"/*")

DIR_LIST.sort()

DIR_LIST

In [ ]:
temp = list(range(20))

In [ ]:
PILL_CLASS = dict(zip(temp, DIR_LIST))

In [ ]:
PILL_CLASS

In [ ]:
for i in range(0, 20) :
    PNG_FILES = glob(PILL_CLASS[i]+"/*.png")

    for png_file in PNG_FILES :
        txt_file = png_file.replace(".png", ".txt")

        f = open(txt_file, 'w')
        f.write(f"{i} 0.5 0.5 0.99 0.99")
        f.close()

In [ ]:
DIR_LIST = glob(PILL_CLASS[0]+"/*.txt")

In [ ]:
DIR_LIST

In [ ]:
import os

os.mkdir(PILL_DATA_DIR)

os.mkdir(PILL_DATA_DIR+"/train")
os.mkdir(PILL_DATA_DIR+"/train/images")
os.mkdir(PILL_DATA_DIR+"/train/labels")

os.mkdir(PILL_DATA_DIR+"/valid")
os.mkdir(PILL_DATA_DIR+"/valid/images")
os.mkdir(PILL_DATA_DIR+"/valid/labels")

os.mkdir(PILL_DATA_DIR+"/test")
os.mkdir(PILL_DATA_DIR+"/test/images")
os.mkdir(PILL_DATA_DIR+"/test/labels")

In [ ]:
# 데이터 섞기
import numpy as np
arr1 = np.arange(1, 100, 10)
arr2 = np.arange(0, 10)

print(arr1)
print(arr2)

np.random.shuffle(arr2)

print(arr1)
print(arr2)

#arr = arr1[arr2]

#print(arr)

In [ ]:
import numpy as np

train_rate = 0.7
valid_rate = 0.1
test_rate = 0.2

PNG_FILES = glob(PILL_CLASS[0]+"/*.png")

total_cnt = len(PNG_FILES)

train_cnt = int(total_cnt * train_rate)
valid_cnt = int(total_cnt * valid_rate)
test_cnt = int(total_cnt * test_rate)

indexes = np.arange(total_cnt)

np.random.shuffle(indexes)

dir_name = PILL_CLASS[0].replace(ORG_DATA_DIR+"/", "")

In [ ]:
for i in range(0, 20) :
    PNG_FILES = glob(PILL_CLASS[i]+"/*.png")

    # 학습용 데이터 셋
    for idx in indexes[:train_cnt] :
        shutil.copy(PNG_FILES[idx], PILL_DATA_DIR+"/train/images")
        shutil.copy(PNG_FILES[idx].replace(".png", ".txt"), PILL_DATA_DIR+"/train/labels")

    # 검증용 데이터 셋
    for idx in indexes[train_cnt:train_cnt+valid_cnt] :
        shutil.copy(PNG_FILES[idx], PILL_DATA_DIR+"/valid/images")
        shutil.copy(PNG_FILES[idx].replace(".png", ".txt"), PILL_DATA_DIR+"/valid/labels")

    # 테스트 데이터 셋
    for idx in indexes[train_cnt+valid_cnt:] :
        shutil.copy(PNG_FILES[idx], PILL_DATA_DIR+"/test/images")
        shutil.copy(PNG_FILES[idx].replace(".png", ".txt"), PILL_DATA_DIR+"/test/labels")

In [ ]:
len(glob(PILL_DATA_DIR + "/train/images/*.png"))

In [ ]:
PILL_CLASS

In [ ]:
import yaml

data = { 'train' : '/content/PILL_data/train/images/',
         'val' : '/content/PILL_data/valid/images/',
         'test' : '/content/PILL_data/test/images',
         'names' : ['K-033829', 'K-034827', 'K-036307', 'K-037578', 'K-038723', 'K-040317', 'K-040339', 'K-041997', 'K-042226', 'K-043499', 'K-043780', 'K-043814', 'K-043935', 'K-045402', 'K-045583', 'K-047218', 'K-049428', 'K-051326', 'K-051672', 'K-052716'],
         'nc' : 20
        }

with open('/content/PILL_data/PILL_Data.yaml', 'w') as f:
  yaml.dump(data, f)


with open('/content/PILL_data/PILL_Data.yaml', 'r') as f:
  PILL_yaml = yaml.safe_load(f)
  display(PILL_yaml)

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
model.train(data='/content/PILL_data/PILL_Data.yaml', epochs=10, patience=30, batch=32, imgsz=416)

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
# 이미지 인식 가능한 경우
import PIL
results = model.predict(source='/content/2023-08-11 103244.png', save=True)

In [ ]:
detetced_image_list = glob('/content/runs/detect/train/**', recursive=True)

detected_image_nums = len(detetced_image_list)

print(detected_image_nums)

print(detetced_image_list)

In [ ]:
# 다운로드를 위한 압축

import zipfile
import os

if not os.path.exists('/content/detected_result_model/'):
    os.mkdir('/content/detected_result_model/')
    print('detected_result dir is created !!!')

with zipfile.ZipFile('/content/detected_result_model/PILL_model.zip', 'w') as detected_images:
    for idx in range(detected_image_nums):
        detected_images.write(detetced_image_list[idx])

In [ ]:
# Load YOLOv8n

from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')  # load a pretrained YOLOv8n detection model

In [ ]:
results = model.predict(source='/content/PILL_data/test/images/K-033829_0_0_1_0_60_200_200.png', save=True)

In [ ]:
results = model.predict(source='/content/그림1.jpg', save=True)